In [4]:
import torch
import random
import numpy as np
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import pandas as pd

RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [28]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
LABELS = [    
    "JUMPING",
    "JUMPING_JACKS",
    "BOXING",
    "WAVING_2HANDS",
    "WAVING_1HAND",
    "CLAPPING_HANDS"] 
DATASET_PATH = "poses/"

X_train_path = "shuffled_x_train_data"
X_test_path = DATASET_PATH + "X_test.txt"

y_train_path = "shuffled_y_train_data"
y_test_path = DATASET_PATH + "Y_test.txt"

# Hyper-parameters 
num_classes = 6
num_epochs = 10
train_batch_size = 200
val_batch_size = 125
test_batch_size = 1
learning_rate = 0.001

input_size = 36
sequence_length = 32
hidden_size = 128
num_layers = 2

In [3]:
device

device(type='cpu')

In [4]:
column_names = [  "j0_x",  "j0_y", "j1_x", "j1_y" , "j2_x", "j2_y", "j3_x", "j3_y", "j4_x", "j4_y", "j5_x", "j5_y", "j6_x", "j6_y", "j7_x", "j7_y", "j8_x", "j8_y", "j9_x", "j9_y", "j10_x", "j10_y", "j11_x", "j11_y", "j12_x", "j12_y", "j13_x", "j13_y", 'j14_x', "j14_y", "j15_x", "j15_y", "j16_x", "j16_y", "j17_x", "j17_y" ]
# x_train_data = pd.read_csv(X_train_path, sep=",", names=column_names, header=None, dtype=np.float32)
shuffled_x_train_data = pd.read_csv(X_train_path, sep="\t", dtype=np.float32)
# x_test_data = pd.read_csv(X_test_path, sep=",", names=column_names, header=None, dtype=np.float32)
# y_train_data = p|d.read_csv(y_train_path, names=["labels"], dtype=np.int_)
shuffled_y_train_data = pd.read_csv(y_train_path, sep="\t", dtype=np.int_)
# y_test_data = pd.read_csv(y_test_path, names=["labels"], dtype=np.int_)

In [5]:
def normalize(data):
    for i, column in enumerate(data):
        if i % 2 == 0:
            data[column] = data[column] / 640
        else:
            data[column] = data[column] / 480
    return data

In [6]:
def unnormalize(data):
    for i, column in enumerate(data):
        if i % 2 == 0:
            data[column] = data[column] * 640
        else:
            data[column] = data[column] * 480
    return data

In [16]:
y_train = shuffled_y_train_data.iloc[:int(shuffled_y_train_data.shape[0]*0.9 - 362)]
x_train = shuffled_x_train_data.iloc[:int(y_train.shape[0]*32)]
x_test = shuffled_x_train_data.iloc[x_train.shape[0]:]
y_test = shuffled_y_train_data.iloc[y_train.shape[0]:]
print(x_train.shape[0]/32)
print(y_train.shape)
print(x_test.shape[0]/32)
print(y_test.shape)

20000.0
(20000, 1)
2625.0
(2625, 1)


In [22]:
def chunker(x_seq, y_seq, seq_size, batch_size):
    for batch_pos in range(0, len(y_seq), batch_size):
        x_batch = list()
        for pos in range(batch_size):
            # print(x_seq.iloc[pos*seq_size:pos*seq_size + seq_size])
            x_batch.append(x_seq.iloc[pos*seq_size:pos*seq_size + seq_size].values)
        
        yield torch.tensor(x_batch), torch.flatten(torch.tensor(y_seq.iloc[batch_pos:batch_pos + batch_size].values))
        

In [27]:
def accuracy(output, target):
    return (output.argmax(dim=1) == target).float().mean().item()

In [25]:
class LSTMClassifier(nn.Module):
    """Very simple implementation of LSTM-based time-series classifier."""
    
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.rnn = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.batch_size = None
        self.hidden = None
    
    def forward(self, x):
        h0, c0 = self.init_hidden(x)
        out, (hn, cn) = self.rnn(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out
    
    def init_hidden(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        return [t.cuda() for t in (h0, c0)]

In [ ]:
input_dim = 10    
hidden_dim = 256
layer_dim = 3
output_dim = 9
seq_dim = 128

lr = 0.0005
n_epochs = 1000
iterations_per_epoch = len(trn_dl)
best_acc = 0
patience, trials = 100, 0

model = LSTMClassifier(inp, hidden_dim, layer_dim, output_dim)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
opt = torch.optim.RMSprop(model.parameters(), lr=lr)
sched = CyclicLR(opt, cosine(t_max=iterations_per_epoch * 2, eta_min=lr/100))

print('Start model training')

for epoch in range(1, n_epochs + 1):
    
    for i, (x_batch, y_batch) in enumerate(trn_dl):
        model.train()
        x_batch = x_batch.cuda()
        y_batch = y_batch.cuda()
        sched.step()
        opt.zero_grad()
        out = model(x_batch)
        loss = criterion(out, y_batch)
        loss.backward()
        opt.step()
    
    model.eval()
    correct, total = 0, 0
    for x_val, y_val in val_dl:
        x_val, y_val = [t.cuda() for t in (x_val, y_val)]
        out = model(x_val)
        preds = F.log_softmax(out, dim=1).argmax(dim=1)
        total += y_val.size(0)
        correct += (preds == y_val).sum().item()
    
    acc = correct / total

    if epoch % 5 == 0:
        print(f'Epoch: {epoch:3d}. Loss: {loss.item():.4f}. Acc.: {acc:2.2%}')

    if acc > best_acc:
        trials = 0
        best_acc = acc
        torch.save(model.state_dict(), 'best.pth')
        print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
    else:
        trials += 1
        if trials >= patience:
            print(f'Early stopping on epoch {epoch}')
            break

In [31]:
# Train the model
n_total_steps = int(y_train.shape[0] / 2)

for epoch in range(num_epochs):
    correct = 0
    for i, (x_batch, y_batch) in enumerate(chunker(x_train, y_train, sequence_length, 200)):  
        # origin shape: [N, 1, 28, 28]
        # resized: [N, 28, 28]
        y_batch = y_batch.to(device)
        x_batch = x_batch.to(device)
        # Forward pass
        outputs = model(x_batch)
        optimizer.zero_grad()
        loss = criterion(outputs, y_batch)
        # Backward and optimize
        
        loss.backward()
        optimizer.step()

        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == y_batch).sum().item()
        if (i+1) % 10 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
    acc = 100.0 * correct / y_train.shape[0]
    print (f'Accuracy: {acc} %')
    # Test the model
    # In test phase, we don't need to compute gradients (for memory efficiency)
    correct = 0
    with torch.no_grad():
        correct = 0
        for i, (x_batch, y_batch) in enumerate(chunker(x_test, y_test, sequence_length, 125)):  
            # origin shape: [N, 1, 28, 28]
            # resized: [N, 28, 28]
            y_batch = y_batch.to(device)
            x_batch = x_batch.to(device)
            outputs = model(x_batch)
            
            # max returns (value ,index)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == y_batch).sum().item()
        acc = 100.0 * correct / y_test.shape[0]
        print(f'Accuracy: {acc} %')

IndexError: Target 2560 is out of bounds.